**Importing libraries**

In [1]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import requests, json

#Ignoring SSL certificates- Reference Dr.Liao's Lecture
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

**Avoiding bot-check before reading into the website**

In [2]:
#using online user-agent to avoid the bots- Reference Dr.Liao's Lecture
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36'}

In [3]:
#Running a loop to generate all 5 metacitric's website links
for i in range(5):  #0-4
    url = 'https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=' + str(i)
    print(url)
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    container = soup.find_all('td', class_ = 'clamp-summary-wrap')
    #print(container)

https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=0
https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=1
https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=2
https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=3
https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=4


In [4]:
def get_name(movie): #Reference from Dr.Liao's Session
    movie_name = movie.find('h3').text #fetching only the movie name from h3 tag using find & text functions
    return movie_name

In [5]:
def get_director(detail_link):  #defining function to get director from detail_link-Reference from Dr.Liao's Session
    detail_page = requests.get(detail_link, headers = headers)
    detail_soup = BeautifulSoup(detail_page.content, "html.parser")
    try:
        director_table = detail_soup.find('table', class_ = 'credits', summary = movie_name + "'s Director Credits")
        director_line = director_table.find_all('td', class_ = 'person')
    except:
        director_table = detail_soup.find('table', class_ = 'credits', summary = movie_name + "' Director Credits")
        director_line = director_table.find_all('td', class_ = 'person')
    directors = list()
    for line in director_line:
        hyperlink = line.find('a')
        director = hyperlink.get_text()
        directors.append(director.strip())
    return directors

In [6]:
def get_cast(detail_link):
    detail_page = requests.get(detail_link, headers = headers)
    detail_soup = BeautifulSoup(detail_page.content, "html.parser")
    try:
        cast_table = detail_soup.find('table', class_ = 'credits', summary = movie_name + "'s Cast Credits" or movie_name + "' Cast Credits")
        p_cast_table = detail_soup.find('table', class_ = 'credits', summary = movie_name + "'s Principal Cast Credits" or movie_name + "' Principal Cast Credits")
    except:
        cast_table = detail_soup.find('table', class_ = 'credits', summary = movie_name + "' Cast Credits")
        cast_line = cast_table.find_all('td', class_ = 'person')
        p_cast_table = detail_soup.find('table', class_ = 'credits', summary = movie_name + "' Principal Cast Credits")
        p_cast_line = p_cast_table.find_all('td', class_ = 'person')
    casts = list()
    q = str(p_cast_table)
    if  q != 'None':
        p_cast_line = p_cast_table.find_all('td', class_ = 'person')
        for line in p_cast_line:
            hyperlink = line.find('a')
            p_cast = hyperlink.get_text()
            casts.append(p_cast.strip())
    p = str(cast_table)
    if  p != 'None':
        cast_line = cast_table.find_all('td', class_ = 'person')
        for line in cast_line:
            hyperlink = line.find('a')
            cast = hyperlink.get_text()
            casts.append(cast.strip())
    elif p == 'None' or q == 'None':
        pass
    return casts

In [7]:
all_movie = []
movie_names=[]
movie_links=[]
movie_dic = {}
import time
n = 0
for i in range(5): #Appending movies, movie links
    url = 'https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=' + str(i)
    #print(url)
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content, "html.parser")
    container = soup.find_all('td', class_ = 'clamp-summary-wrap')
    for movie in container: #loop through each of the 500 movies
        n = n + 1
        print(n)
        #print(movie)
        movie_name = get_name(movie) #calling get_name() func
        print(movie_name)
        movie_names.append(movie_name) #creating a list for all movie names
        movie_link_tag = movie.find('a', class_ = 'title') #then lets get movie link
        movie_link = movie_link_tag.get('href', None) # note we are still in the for loop, so we are only dealing with one movie at a time
        movie_link = 'https://www.metacritic.com'+ movie_link
        #print(movie_link)
        movie_links.append(movie_link) #creating a list for all movie links
        detail_page = requests.get(movie_link, headers = headers)
        detail_soup = BeautifulSoup(detail_page.content, "html.parser")
        div = detail_soup('div', class_= 'see_all pad_btm1')
        for line in div:
            hyperlink = line.find('a')
            detail_link = hyperlink.get('href', None) 
            detail_link = 'https://www.metacritic.com'+ detail_link 
            print(detail_link)
            director = get_director(detail_link) #calling get_director() func
            print(director)
            cast = get_cast(detail_link) #calling get_cast() function
            time.sleep(1)
        dir_dic = {} #Reference- Dr.Liao's Code
        dir_dic[tuple(director)] = cast #converting director into a tuple, becasue it was a list
        print(dir_dic)
        movie_dic[movie_name] = dir_dic 
        all_movie.append(movie_dic) #converting movie_dic to a list
print(movie_dic)

1
Citizen Kane
https://www.metacritic.com/movie/citizen-kane-1941/details
['Orson Welles']
{('Orson Welles',): ['Joseph Cotten', 'Orson Welles', 'Agnes Moorehead', 'Dorothy Comingore', 'Erskine Sanford', 'Everett Sloane', 'Fortunio Bonanova', 'George Coulouris', 'Georgia Backus', 'Gus Schilling', 'Harry Shannon', 'Paul Stewart', 'Philip Van Zandt', 'Ray Collins', 'Ruth Warrick', 'William Alland']}
2
The Godfather
https://www.metacritic.com/movie/the-godfather/details
['Francis Ford Coppola']
{('Francis Ford Coppola',): ['Al Pacino', 'Marlon Brando', 'Abe Vigoda', 'Al Lettieri', 'Diane Keaton', 'Gianni Russo', 'James Caan', 'John Cazale', 'John Marley', 'Richard Conte', 'Richard S. Castellano', 'Robert Duvall', 'Rudy Bond', 'Sterling Hayden', 'Talia Shire']}
3
Rear Window
https://www.metacritic.com/movie/rear-window/details
['Alfred Hitchcock']
{('Alfred Hitchcock',): ['Frank Cady', 'Georgine Darcy', 'Grace Kelly', 'James Stewart', 'Judith Evelyn', 'Raymond Burr', 'Ross Bagdasarian', 'S

In [8]:
print(type(movie_dic)) #A dictionary
all_movie = []
all_movie.append(movie_dic) #Converting dict to list
print(all_movie)
print(type(all_movie)) # A list
print(len(all_movie)) #length is 1


<class 'dict'>
[{'Citizen Kane': {('Orson Welles',): ['Joseph Cotten', 'Orson Welles', 'Agnes Moorehead', 'Dorothy Comingore', 'Erskine Sanford', 'Everett Sloane', 'Fortunio Bonanova', 'George Coulouris', 'Georgia Backus', 'Gus Schilling', 'Harry Shannon', 'Paul Stewart', 'Philip Van Zandt', 'Ray Collins', 'Ruth Warrick', 'William Alland']}, 'The Godfather': {('Francis Ford Coppola',): ['Al Pacino', 'Marlon Brando', 'Abe Vigoda', 'Al Lettieri', 'Diane Keaton', 'Gianni Russo', 'James Caan', 'John Cazale', 'John Marley', 'Richard Conte', 'Richard S. Castellano', 'Robert Duvall', 'Rudy Bond', 'Sterling Hayden', 'Talia Shire']}, 'Rear Window': {('Alfred Hitchcock',): ['Frank Cady', 'Georgine Darcy', 'Grace Kelly', 'James Stewart', 'Judith Evelyn', 'Raymond Burr', 'Ross Bagdasarian', 'Sara Berner', 'Thelma Ritter', 'Wendell Corey', 'Havis Davenport', 'Irene Winston', 'Jerry Antes', 'Jesslyn Fax', 'Rand Harper']}, 'Casablanca': {('Michael Curtiz',): ['Humphrey Bogart', 'Ingrid Bergman', 'Cla

In [9]:
print(len(movie_dic)) #length is 500
print(movie_dic)


{'Citizen Kane': {('Orson Welles',): ['Joseph Cotten', 'Orson Welles', 'Agnes Moorehead', 'Dorothy Comingore', 'Erskine Sanford', 'Everett Sloane', 'Fortunio Bonanova', 'George Coulouris', 'Georgia Backus', 'Gus Schilling', 'Harry Shannon', 'Paul Stewart', 'Philip Van Zandt', 'Ray Collins', 'Ruth Warrick', 'William Alland']}, 'The Godfather': {('Francis Ford Coppola',): ['Al Pacino', 'Marlon Brando', 'Abe Vigoda', 'Al Lettieri', 'Diane Keaton', 'Gianni Russo', 'James Caan', 'John Cazale', 'John Marley', 'Richard Conte', 'Richard S. Castellano', 'Robert Duvall', 'Rudy Bond', 'Sterling Hayden', 'Talia Shire']}, 'Rear Window': {('Alfred Hitchcock',): ['Frank Cady', 'Georgine Darcy', 'Grace Kelly', 'James Stewart', 'Judith Evelyn', 'Raymond Burr', 'Ross Bagdasarian', 'Sara Berner', 'Thelma Ritter', 'Wendell Corey', 'Havis Davenport', 'Irene Winston', 'Jerry Antes', 'Jesslyn Fax', 'Rand Harper']}, 'Casablanca': {('Michael Curtiz',): ['Humphrey Bogart', 'Ingrid Bergman', 'Claude Rains', 'Con

In [10]:
print(len(all_movie)) #length is 1
print(all_movie)


1
[{'Citizen Kane': {('Orson Welles',): ['Joseph Cotten', 'Orson Welles', 'Agnes Moorehead', 'Dorothy Comingore', 'Erskine Sanford', 'Everett Sloane', 'Fortunio Bonanova', 'George Coulouris', 'Georgia Backus', 'Gus Schilling', 'Harry Shannon', 'Paul Stewart', 'Philip Van Zandt', 'Ray Collins', 'Ruth Warrick', 'William Alland']}, 'The Godfather': {('Francis Ford Coppola',): ['Al Pacino', 'Marlon Brando', 'Abe Vigoda', 'Al Lettieri', 'Diane Keaton', 'Gianni Russo', 'James Caan', 'John Cazale', 'John Marley', 'Richard Conte', 'Richard S. Castellano', 'Robert Duvall', 'Rudy Bond', 'Sterling Hayden', 'Talia Shire']}, 'Rear Window': {('Alfred Hitchcock',): ['Frank Cady', 'Georgine Darcy', 'Grace Kelly', 'James Stewart', 'Judith Evelyn', 'Raymond Burr', 'Ross Bagdasarian', 'Sara Berner', 'Thelma Ritter', 'Wendell Corey', 'Havis Davenport', 'Irene Winston', 'Jerry Antes', 'Jesslyn Fax', 'Rand Harper']}, 'Casablanca': {('Michael Curtiz',): ['Humphrey Bogart', 'Ingrid Bergman', 'Claude Rains', '

In [11]:
print(len(movie_names)) #Length is 500
print(movie_names)


500
['Citizen Kane', 'The Godfather', 'Rear Window', 'Casablanca', 'Boyhood', 'Three Colors: Red', 'Vertigo', 'Notorious', "Singin' in the Rain", 'City Lights', 'Moonlight', 'Intolerance', 'Pinocchio', 'Touch of Evil', 'The Lady Vanishes', 'The Treasure of the Sierra Madre', "Pan's Labyrinth", 'Some Like It Hot', 'North by Northwest', 'Hoop Dreams', 'Rashomon', 'All About Eve', 'Jules and Jim', 'The Wild Bunch', 'My Left Foot', 'The Third Man', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb', 'Quo Vadis, Aida?', 'Gone with the Wind', '4 Months, 3 Weeks and 2 Days', 'Psycho', 'Battleship Potemkin', 'A Streetcar Named Desire', 'American Graffiti', 'Dumbo', 'Roma', 'Ran', 'The Shop Around the Corner', '12 Angry Men', 'Manchester by the Sea', "Rosemary's Baby", 'The Maltese Falcon', '12 Years a Slave', 'Killer of Sheep', 'Nashville', 'Ratatouille', 'Parasite', 'Rocks', "Don't Look Now", 'The Grapes of Wrath', 'Children of Paradise (1945)', 'Mean Streets', 'The Lady E

**User Input**

In [12]:
inp_movie = input('Which movie do you want to check from the list? ')
input2 = input('What information about this movie do you want to check? (Choose director or cast)')

while(True):#implementing a while loop to handle invalid inputs
        try:
            if inp_movie in movie_names: #looking in movie_names list for movies
                break
            else:
                print('Movie name is not in the Top 500 list' )
                inp_movie = input('Please enter a movie which is in Top 500 from the website ')
                continue
        except: 
            print('Invalid Input')
            break
print(inp_movie)

while(True): #implementing a while loop to handle invalid inputs for director/cast
    try:
        if (input2 == 'Director') or (input2 == 'Cast') or (input2 == 'director') or (input2 == 'cast'):#checking for 2 types of input for both director and cast
            break
        else:
            print('Wrong Input.')
            input2 = input('What information about this movie do you want to check? (Choose director or cast)')
            continue
    except:
        print('Invalid Input')
        break

for each in dir_dic:
    for movie_key, movie_value in movie_dic.items():
        if inp_movie == movie_key:
            if input2 == 'director' or input2 == 'Director': #considering two input types for director
                for sub_key, sub_value in movie_value.items():
                    direct = ''
                    for i in sub_key:
                        direct = direct + i + ','
                    print('The director of the movie', inp_movie, 'is: \n', direct)
            elif input2 =='cast' or input2 =='Cast': #considering two input types for cast
                for sub_key, sub_value in movie_value.items():
                    cast2 = ''
                    for i in sub_value:
                        cast2 = cast2 + i + ','
                    print('The cast of the movie',inp_movie, 'includes: \n',cast2)

Which movie do you want to check from the list? Notorious
What information about this movie do you want to check? (Choose director or cast)cast
Notorious
The cast of the movie Notorious includes: 
 


In [13]:
p = [] #creating a list for all movie names by taking the key value of movie_dic
for each in all_movie:
    for movie_key, movie_value in movie_dic.items():
        p.append(movie_key) 
        #print(movie_key)
        #print(movie_value)
print(len(p)) #Length is 497
print(p)

497
['Citizen Kane', 'The Godfather', 'Rear Window', 'Casablanca', 'Boyhood', 'Three Colors: Red', 'Vertigo', 'Notorious', "Singin' in the Rain", 'City Lights', 'Moonlight', 'Intolerance', 'Pinocchio', 'Touch of Evil', 'The Lady Vanishes', 'The Treasure of the Sierra Madre', "Pan's Labyrinth", 'Some Like It Hot', 'North by Northwest', 'Hoop Dreams', 'Rashomon', 'All About Eve', 'Jules and Jim', 'The Wild Bunch', 'My Left Foot', 'The Third Man', 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb', 'Quo Vadis, Aida?', 'Gone with the Wind', '4 Months, 3 Weeks and 2 Days', 'Psycho', 'Battleship Potemkin', 'A Streetcar Named Desire', 'American Graffiti', 'Dumbo', 'Roma', 'Ran', 'The Shop Around the Corner', '12 Angry Men', 'Manchester by the Sea', "Rosemary's Baby", 'The Maltese Falcon', '12 Years a Slave', 'Killer of Sheep', 'Nashville', 'Ratatouille', 'Parasite', 'Rocks', "Don't Look Now", 'The Grapes of Wrath', 'Children of Paradise (1945)', 'Mean Streets', 'The Lady E

In [14]:
print(type(movie_key))
print(type(movie_value))

<class 'str'>
<class 'dict'>


In [15]:
sub_dict = list(movie_dic.values())
print(sub_dict)
for i in sub_dict:
    print(type(i))

[{('Orson Welles',): ['Joseph Cotten', 'Orson Welles', 'Agnes Moorehead', 'Dorothy Comingore', 'Erskine Sanford', 'Everett Sloane', 'Fortunio Bonanova', 'George Coulouris', 'Georgia Backus', 'Gus Schilling', 'Harry Shannon', 'Paul Stewart', 'Philip Van Zandt', 'Ray Collins', 'Ruth Warrick', 'William Alland']}, {('Francis Ford Coppola',): ['Al Pacino', 'Marlon Brando', 'Abe Vigoda', 'Al Lettieri', 'Diane Keaton', 'Gianni Russo', 'James Caan', 'John Cazale', 'John Marley', 'Richard Conte', 'Richard S. Castellano', 'Robert Duvall', 'Rudy Bond', 'Sterling Hayden', 'Talia Shire']}, {('Alfred Hitchcock',): ['Frank Cady', 'Georgine Darcy', 'Grace Kelly', 'James Stewart', 'Judith Evelyn', 'Raymond Burr', 'Ross Bagdasarian', 'Sara Berner', 'Thelma Ritter', 'Wendell Corey', 'Havis Davenport', 'Irene Winston', 'Jerry Antes', 'Jesslyn Fax', 'Rand Harper']}, {('Michael Curtiz',): ['Humphrey Bogart', 'Ingrid Bergman', 'Claude Rains', 'Conrad Veidt', 'Curt Bois', 'Dooley Wilson', 'Enrique Acosta', 'J

In [16]:
print(len(sub_dict))#Length is 497

497


In [17]:
all_directors = tuple()
k = 0
while (k < len(movie_dic)):
    for i in all_movie:
        sub = list(movie_dic.values())[k]
        #type of movie_director.values() is dict values, so converting them to a list
        sub_key = list(sub.keys())
        for tpl in sub_key:
            all_directors += tpl
    k = k + 1
print(len(all_directors))#Length is 588
print(all_directors)


588
('Orson Welles', 'Francis Ford Coppola', 'Alfred Hitchcock', 'Michael Curtiz', 'Richard Linklater', 'Krzysztof Kieslowski', 'Alfred Hitchcock', 'Alfred Hitchcock', 'Gene Kelly', 'Stanley Donen', 'Charles Chaplin', 'Barry Jenkins', 'D.W. Griffith', 'Ben Sharpsteen', 'Bill Roberts', 'Hamilton Luske', 'Jack Kinney', 'Norman Ferguson', 'T. Hee', 'Wilfred Jackson', 'Orson Welles', 'Alfred Hitchcock', 'John Huston', 'Guillermo del Toro', 'Billy Wilder', 'Alfred Hitchcock', 'Steve James', 'Akira Kurosawa', 'Joseph L. Mankiewicz', 'François Truffaut', 'Sam Peckinpah', 'Jim Sheridan', 'Carol Reed', 'Stanley Kubrick', 'Jasmila Zbanic', 'George Cukor', 'Sam Wood', 'Victor Fleming', 'Cristian Mungiu', 'Alfred Hitchcock', 'Sergei M. Eisenstein', 'Elia Kazan', 'George Lucas', 'Ben Sharpsteen', 'Bill Roberts', 'Jack Kinney', 'John Elliotte', 'Norman Ferguson', 'Samuel Armstrong', 'Wilfred Jackson', 'Alfonso Cuarón', 'Akira Kurosawa', 'Ernst Lubitsch', 'Sidney Lumet', 'Kenneth Lonergan', 'Roman Po

In [18]:
all_director_lst = list(all_directors)
dictor_count = dict()
for i in all_director_lst:
    dictor_count[i] = dictor_count.get(i, 0) + 1 
print(dictor_count)

{'Orson Welles': 5, 'Francis Ford Coppola': 5, 'Alfred Hitchcock': 12, 'Michael Curtiz': 4, 'Richard Linklater': 3, 'Krzysztof Kieslowski': 2, 'Gene Kelly': 1, 'Stanley Donen': 1, 'Charles Chaplin': 2, 'Barry Jenkins': 2, 'D.W. Griffith': 1, 'Ben Sharpsteen': 4, 'Bill Roberts': 4, 'Hamilton Luske': 2, 'Jack Kinney': 2, 'Norman Ferguson': 3, 'T. Hee': 2, 'Wilfred Jackson': 4, 'John Huston': 5, 'Guillermo del Toro': 2, 'Billy Wilder': 3, 'Steve James': 3, 'Akira Kurosawa': 2, 'Joseph L. Mankiewicz': 1, 'François Truffaut': 2, 'Sam Peckinpah': 1, 'Jim Sheridan': 1, 'Carol Reed': 2, 'Stanley Kubrick': 5, 'Jasmila Zbanic': 1, 'George Cukor': 4, 'Sam Wood': 1, 'Victor Fleming': 1, 'Cristian Mungiu': 1, 'Sergei M. Eisenstein': 1, 'Elia Kazan': 2, 'George Lucas': 2, 'John Elliotte': 1, 'Samuel Armstrong': 3, 'Alfonso Cuarón': 3, 'Ernst Lubitsch': 1, 'Sidney Lumet': 2, 'Kenneth Lonergan': 1, 'Roman Polanski': 2, 'Steve McQueen': 4, 'Charles Burnett': 1, 'Robert Altman': 4, 'Brad Bird': 2, 'Joon

In [19]:
all_casts = list()
m = 0
while (m < len(movie_dic)):
    for i in all_movie:
        sub = list(movie_dic.values())[m]
        sub_key = list(sub.values())
        for tpl in sub_key:
            all_casts += tpl
    m = m + 1
print(all_casts)
print(len(all_casts))

['Joseph Cotten', 'Orson Welles', 'Agnes Moorehead', 'Dorothy Comingore', 'Erskine Sanford', 'Everett Sloane', 'Fortunio Bonanova', 'George Coulouris', 'Georgia Backus', 'Gus Schilling', 'Harry Shannon', 'Paul Stewart', 'Philip Van Zandt', 'Ray Collins', 'Ruth Warrick', 'William Alland', 'Al Pacino', 'Marlon Brando', 'Abe Vigoda', 'Al Lettieri', 'Diane Keaton', 'Gianni Russo', 'James Caan', 'John Cazale', 'John Marley', 'Richard Conte', 'Richard S. Castellano', 'Robert Duvall', 'Rudy Bond', 'Sterling Hayden', 'Talia Shire', 'Frank Cady', 'Georgine Darcy', 'Grace Kelly', 'James Stewart', 'Judith Evelyn', 'Raymond Burr', 'Ross Bagdasarian', 'Sara Berner', 'Thelma Ritter', 'Wendell Corey', 'Havis Davenport', 'Irene Winston', 'Jerry Antes', 'Jesslyn Fax', 'Rand Harper', 'Humphrey Bogart', 'Ingrid Bergman', 'Claude Rains', 'Conrad Veidt', 'Curt Bois', 'Dooley Wilson', 'Enrique Acosta', 'John Qualen', 'Joy Page', 'Leonid Kinskey', 'Madeleine Lebeau', 'Paul Henreid', 'Peter Lorre', 'S.Z. Saka

In [20]:
all_cast_lst = list(all_casts)
cast_count = dict()
for i in all_cast_lst:
    cast_count[i] = cast_count.get(i, 0) + 1 #using count func of dictionaries
print(cast_count)

{'Joseph Cotten': 4, 'Orson Welles': 5, 'Agnes Moorehead': 1, 'Dorothy Comingore': 1, 'Erskine Sanford': 1, 'Everett Sloane': 1, 'Fortunio Bonanova': 2, 'George Coulouris': 1, 'Georgia Backus': 2, 'Gus Schilling': 1, 'Harry Shannon': 2, 'Paul Stewart': 2, 'Philip Van Zandt': 1, 'Ray Collins': 2, 'Ruth Warrick': 1, 'William Alland': 1, 'Al Pacino': 4, 'Marlon Brando': 7, 'Abe Vigoda': 1, 'Al Lettieri': 1, 'Diane Keaton': 3, 'Gianni Russo': 1, 'James Caan': 3, 'John Cazale': 2, 'John Marley': 1, 'Richard Conte': 1, 'Richard S. Castellano': 1, 'Robert Duvall': 5, 'Rudy Bond': 4, 'Sterling Hayden': 4, 'Talia Shire': 2, 'Frank Cady': 1, 'Georgine Darcy': 1, 'Grace Kelly': 2, 'James Stewart': 6, 'Judith Evelyn': 1, 'Raymond Burr': 1, 'Ross Bagdasarian': 1, 'Sara Berner': 1, 'Thelma Ritter': 2, 'Wendell Corey': 1, 'Havis Davenport': 1, 'Irene Winston': 1, 'Jerry Antes': 1, 'Jesslyn Fax': 1, 'Rand Harper': 1, 'Humphrey Bogart': 5, 'Ingrid Bergman': 1, 'Claude Rains': 2, 'Conrad Veidt': 1, 'Cur

In [21]:
print(type(all_cast_lst))
print(len(all_cast_lst))

<class 'list'>
6907


In [23]:
all_casts = list()
m = 0
listToStr = []
castlist = []
while (m < len(movie_dic)):
    for i in all_movie:
        sub = list(movie_dic.values())[m] #so this extracts the sub dictionary
        sub_key = list(sub.values())
        for tpl in sub_key:
            all_casts = tpl
            #print(all_casts)
            listToStr = ':'.join(map(str, all_casts)) #joining all the cast values into one string
            castlist.append(listToStr)         
    m = m + 1
print(castlist)
print(type(castlist))


['Joseph Cotten:Orson Welles:Agnes Moorehead:Dorothy Comingore:Erskine Sanford:Everett Sloane:Fortunio Bonanova:George Coulouris:Georgia Backus:Gus Schilling:Harry Shannon:Paul Stewart:Philip Van Zandt:Ray Collins:Ruth Warrick:William Alland', 'Al Pacino:Marlon Brando:Abe Vigoda:Al Lettieri:Diane Keaton:Gianni Russo:James Caan:John Cazale:John Marley:Richard Conte:Richard S. Castellano:Robert Duvall:Rudy Bond:Sterling Hayden:Talia Shire', 'Frank Cady:Georgine Darcy:Grace Kelly:James Stewart:Judith Evelyn:Raymond Burr:Ross Bagdasarian:Sara Berner:Thelma Ritter:Wendell Corey:Havis Davenport:Irene Winston:Jerry Antes:Jesslyn Fax:Rand Harper', 'Humphrey Bogart:Ingrid Bergman:Claude Rains:Conrad Veidt:Curt Bois:Dooley Wilson:Enrique Acosta:John Qualen:Joy Page:Leonid Kinskey:Madeleine Lebeau:Paul Henreid:Peter Lorre:S.Z. Sakall:Sydney Greenstreet', 'Bonnie Cross:Elijah Smith:Ellar Coltrane:Ethan Hawke:Libby Villari:Lorelei Linklater:Marco Perella:Patricia Arquette:Steven Chester Prince:Alys

In [24]:
all_directors = tuple() #creating an empty tuple
k = 0
dirlist =[] #creating an empty list
while (k < len(movie_dic)):
    for i in all_movie:
        sub = list(movie_dic.values())[k] #extracting the sub dictionary
        sub_key = list(sub.keys())
        for tpl in sub_key:
            all_directors += tpl
            listToStr = ':'.join(map(str, sub_key))
            listToStr = listToStr.replace(',', '')
            listToStr = listToStr.replace('(', '')
            listToStr = listToStr.replace(')', '')
            listToStr = listToStr.replace("'", "")
            dirlist.append(listToStr)
    k = k + 1
print(dirlist)
print(type(dirlist))

['Orson Welles', 'Francis Ford Coppola', 'Alfred Hitchcock', 'Michael Curtiz', 'Richard Linklater', 'Krzysztof Kieslowski', 'Alfred Hitchcock', 'Alfred Hitchcock', 'Gene Kelly Stanley Donen', 'Charles Chaplin', 'Barry Jenkins', 'D.W. Griffith', 'Ben Sharpsteen Bill Roberts Hamilton Luske Jack Kinney Norman Ferguson T. Hee Wilfred Jackson', 'Orson Welles', 'Alfred Hitchcock', 'John Huston', 'Guillermo del Toro', 'Billy Wilder', 'Alfred Hitchcock', 'Steve James', 'Akira Kurosawa', 'Joseph L. Mankiewicz', 'François Truffaut', 'Sam Peckinpah', 'Jim Sheridan', 'Carol Reed', 'Stanley Kubrick', 'Jasmila Zbanic', 'George Cukor Sam Wood Victor Fleming', 'Cristian Mungiu', 'Alfred Hitchcock', 'Sergei M. Eisenstein', 'Elia Kazan', 'George Lucas', 'Ben Sharpsteen Bill Roberts Jack Kinney John Elliotte Norman Ferguson Samuel Armstrong Wilfred Jackson', 'Alfonso Cuarón', 'Akira Kurosawa', 'Ernst Lubitsch', 'Sidney Lumet', 'Kenneth Lonergan', 'Roman Polanski', 'John Huston', 'Steve McQueen', 'Charles

**Creating Pandas Dataframe for Movies**

In [25]:
import pandas as pd

data = {"Movies":p}
Movies = pd.DataFrame.from_dict(data)
Movies

,Movies
0,Citizen Kane
1,The Godfather
2,Rear Window
3,Casablanca
4,Boyhood
...,...
492,Room
493,The Tillman Story
494,The Wind Will Carry Us
495,The Remains of the Day


In [26]:
import pandas as pd
import csv

data = {'Film': p, 'Director': dirlist, 'Cast': castlist} #Creating 3 columns in the data frame
Movies = pd.DataFrame.from_dict(data)
  
Movies

,Film,Director,Cast
0,Citizen Kane,Orson Welles,Joseph Cotten:Orson Welles:Agnes Moorehead:Dor...
1,The Godfather,Francis Ford Coppola,Al Pacino:Marlon Brando:Abe Vigoda:Al Lettieri...
2,Rear Window,Alfred Hitchcock,Frank Cady:Georgine Darcy:Grace Kelly:James St...
3,Casablanca,Michael Curtiz,Humphrey Bogart:Ingrid Bergman:Claude Rains:Co...
4,Boyhood,Richard Linklater,Bonnie Cross:Elijah Smith:Ellar Coltrane:Ethan...
...,...,...,...
492,Room,Lenny Abrahamson,Brie Larson:Jacob Tremblay:Amanda Brugel:Brad ...
493,The Tillman Story,Amir Bar-Lev,Pat Tillman:Pat Tillman:Richard Tillman
494,The Wind Will Carry Us,Abbas Kiarostami,
495,The Remains of the Day,James Ivory,Anthony Hopkins:Emma Thompson:Abigail Hopkins:...


In [27]:
Ramya_df = pd.DataFrame(list(zip(movie_names,movie_links,dirlist,castlist)))

In [28]:
Ramya_df.shape

(497, 4)

**Creating a CSV File download link**

In [29]:
# importing the modules
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "Ramya_Movies.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a random sample dataframe

# create a link to download the dataframe
create_download_link(Ramya_df)

**Creation of SQL Table**

In [30]:
import sqlite3

#writing to csv file
Ramya_df.to_csv("Ramya_Movies.csv")

#Creating MovieInfoDatabase
conn = sqlite3.connect('MovieInfoDatabase.db')
Ramya_df.applymap(str).to_sql('movies', conn, if_exists='replace', index=False)


conn = sqlite3.connect('MovieInfoDatabase.db')
movies_test = pd.read_sql('SELECT * FROM movies', conn)
display(movies_test.head())

,0,1,2,3
0,Citizen Kane,https://www.metacritic.com/movie/citizen-kane,Orson Welles,Joseph Cotten:Orson Welles:Agnes Moorehead:Dor...
1,The Godfather,https://www.metacritic.com/movie/the-godfather,Francis Ford Coppola,Al Pacino:Marlon Brando:Abe Vigoda:Al Lettieri...
2,Rear Window,https://www.metacritic.com/movie/rear-window,Alfred Hitchcock,Frank Cady:Georgine Darcy:Grace Kelly:James St...
3,Casablanca,https://www.metacritic.com/movie/casablanca,Michael Curtiz,Humphrey Bogart:Ingrid Bergman:Claude Rains:Co...
4,Boyhood,https://www.metacritic.com/movie/boyhood,Richard Linklater,Bonnie Cross:Elijah Smith:Ellar Coltrane:Ethan...
